In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from datetime import datetime
import base64
import hashlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from typing import List, Dict
from typing import Callable

# Caminho para o arquivo xlsx local
file_path = './banco_dados.xlsx'

In [2]:
# Funções de conversão
def substituir_string(texto, lista_substituicoes):
    for antigo, novo in lista_substituicoes:
        texto = texto.replace(antigo, novo)
    return texto

def string_to_unique_float(s: str) -> float:
    # Gera um hash MD5 da string
    md5_hash = hashlib.md5(s.encode()).hexdigest()
    # Converte o hash em um número inteiro
    int_value = int(md5_hash, 16)
    # Normaliza o valor para o intervalo entre [0,1] e retorna
    return int_value % (10**8) / (10**8)

def replace_with_other(dataResp: Dict[str, list[str]],colNum: int, text: str,list: List,func: Callable[[str], str] = string_to_unique_float):
    text = substituir_string(text,list)
    if text.startswith("other:") :
        appendData(dataResp, f'Resp_{colNum}', func(text.replace("other:","")))
    else:
        appendData(dataResp, f'Resp_{colNum}', text)  

def appendData(dataResp: Dict[str, list[str]], key: str, colData: str):
    try:
        dataResp[key].append(colData)
    except KeyError:
        dataResp[key] = [colData]


def trataCol(dataResp: Dict[str, list[str]], colNum: int, colData: str, timeAtual: pd.Timestamp, timeAnterior: pd.Timestamp):
    timeElipsed = (timeAtual - timeAnterior).total_seconds()    
    colData = colData.replace("other;", "other:").replace("Sucess;", "").replace("Sucess", "")    
    match colNum:
        case 2:
            resp = []
            for index,item in enumerate(colData.split(";")):
                match index:
                    case 0: # Que horas são neste instante?
                        hora = item
                    case 1: # Data de hoje?
                        # Calcula o quanto o entrevistado estava errado em noção do tempo
                        appendData(dataResp, 'Resp_1', (timeAtual - pd.to_datetime(f"{item} {hora}", dayfirst=True)).total_seconds())
                    case 2: # Qual a sua Idade?
                        appendData(dataResp, 'Resp_2', item)                       
                    case 3: # Gênero *
                        replace_with_other(dataResp,index,item,[("Feminino", "0"), ("Masculino", "1")])                         
                    case 4: # Qual foi o sexo atribuído no seu nascimento?
                        replace_with_other(dataResp,index,item,[("Feminino", "0"), ("Masculino", "1")])
                    case 5: # Assinale a alternativa que identifica a sua Cor ou Raça:
                        appendData(dataResp, f'Resp_{index}', string_to_unique_float(item.replace("other:","")))                        
                        # replace_with_other(dataResp,index,item,[("Preta", "0"), ("Branca", "1"), ("Parda", "2"), ("Amarela", "3"), ("IndÍgena", "4")])
                    case 6: # Dentro de sua família, você é o(a) único(a) filho(a)?
                        replace_with_other(dataResp,index,item,[("Sim", "0")],lambda x: x)
                    case 7: # Qual o seu estado civil?
                        appendData(dataResp, f'Resp_{index}', string_to_unique_float(item.replace("other:","")))   
                    case 8: # Possui filhos(as)?
                        replace_with_other(dataResp,index,item,[("Sim", "0")],lambda x: x) 
                    case 9: # Possui filhos(as) menores de 6 anos?
                        replace_with_other(dataResp,index,item,[("Não", "0"), ("Sim", "1")])    
                    case 10: # Religião *
                        replace_with_other(dataResp,index,item,[("Sem religião", "0")])
                    case 11: # Escolaridade *
                        replace_with_other(dataResp,index,item,[("Sem Escolaridade", "0.0"),("Ensino Fundamental (1º grau) incompleto", "0.1"),("Ensino Fundamental (1º grau) completo", "0.2"),("Ensino Médio (2º grau) incompleto", "0.3"),("Ensino Médio (2º grau) completo", "0.4"),("Superior Incompleto", "0.55"),("Superior Completo", "0.7"),("Mestrado", "0.85"),("Doutorado", "1")])
                    case 12: # Renda familiar mensal de sua casa (somatória)
                        replace_with_other(dataResp,index,item,[("Até 1 salário mínimo", "0.0"),("Mais de 1 a 2 salários mínimos", "0.15"),("Mais de 2 a 3 salários mínimos", "0.3"),("Mais de 3 a 5 salários mínimos", "0.45"),("Mais de 5 a 8 salários mínimos", "0.6"),("Mais de 8 a 12 salários mínimos", "0.75"),("Mais de 12 a 20 salários mínimos", "9.0"),("Mais de 20 salários mínimos", "1.0")])                     
                appendData(dataResp, f'Elipsed_{index}', timeElipsed)                            
            return resp
        case 3:  
            replace_with_other(dataResp,colNum,colData,[(";", ""),("Não", "0")])
            appendData(dataResp, f'Elipsed_{colNum}', timeElipsed)                               
        case _:
            appendData(dataResp, f'Resp_N{colNum}', re.sub(r'<optimized out>#\w+\((Sim|Não)\)', r'\1', colData))
            appendData(dataResp, f'Elipsed_N{colNum}', timeElipsed)                
            return [colData]

In [3]:
# Leitura do arquivo xlsx usando pandas
df = pd.read_excel(file_path, sheet_name='BDados')

# Inicializa variáveis necessárias
dataResp = {}
count = 0
timeAnterior = []  # Inicializa timeAnterior como uma lista vazia

# Loop para processar colunas que começam com "Tela"
for indexCol, colStr in enumerate(df.columns):

    if colStr.startswith("Tela") and df.iloc[0, indexCol] != "" and pd.notna(df.iloc[0, indexCol]):
        timeAtual = []
        timeElipsed = []
        value = []
        
        colNum = int(re.search(r"Tela (\d+)", colStr).group(1))
        
        for indexRow in range(df.shape[0]):
            cell_value = str(df.iloc[indexRow, indexCol])
            match = re.match(r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}) - (.+)", cell_value)
            
            if match:
                timeAtual.append(pd.to_datetime(match.group(1)))
                if colNum != 1:                                    
                    trataCol(dataResp, colNum, match.group(2), timeAtual[-1], timeAnterior[indexRow]) # globals()[f"func_tela_colNum"](match.group(2))
            else:
                print("Ocorreu um erro ao processar a célula:", cell_value)
        
        timeAnterior = timeAtual 
        
# Cria o novo DataFrame com os dados processados
df_novo = pd.DataFrame(dataResp)

# Usando ExcelWriter para adicionar a nova aba ao arquivo Excel existente com if_sheet_exists='replace'
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_novo.to_excel(writer, sheet_name='TDados', index=False)

ValueError: All arrays must be of the same length